In [1]:
import os
import django
import requests
import datetime
import json
from tqdm import tqdm
import warnings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
warnings.filterwarnings(action='ignore')

In [2]:
import re
import requests
import pickle
from bs4 import BeautifulSoup as bs
from collections import Counter, OrderedDict

In [3]:
# https://developers.themoviedb.org/3/discover/movie-discover
URL = 'https://api.themoviedb.org/3'
API_KEY = os.environ.get('MOVIE_KEY')

## 장르

In [4]:
url_genres = f'{URL}/genre/movie/list?api_key={API_KEY}'
result = requests.get(url_genres)
genres = result.json()['genres']
for g in genres:
    genre = Genre()
    genre.pk = g['id']
    genre.genre_name = g['name']
    genre.save()

## 영화(2000~2021)

In [6]:
year = 2020

In [9]:
with open(f'../data/movie_{year}.json', 'r') as fp:
    movies = json.load(fp)

# delete stored movies
movies_year = Movie.objects.filter(release_date__year=year)
movies_year.delete()
# using bulk
to_db_movies = []
for mv in tqdm(movies):
    movie = Movie()
    movie.pk = mv['id']
    movie.title_en = mv['original_title']
    movie.title_ko = mv['title']
    movie.rate = mv['vote_average']
    movie.rate_people_count = mv['vote_count']
    movie.poster_path = mv['poster_path']
    movie.description = mv['overview']
    movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
    to_db_movies.append(movie)

100%|█████████████████████████████████████████████████████████████████████| 24309/24309 [00:00<00:00, 34685.52it/s]


In [13]:
div = 100
to_db_movies_batch = []
for i in range(len(to_db_movies) // div + 1):
    to_db_movies_batch.append(to_db_movies[i*div:(i+1)*div])

In [38]:
def store_movies(year):
    print(f'storing-----------{year}')
    with open(f'../data/movie_{year}.json', 'r') as fp:
        movies = json.load(fp)
    
    # delete stored movies
    movies_year = Movie.objects.filter(release_date__year=year)
    movies_year.delete()
    # using bulk
    to_db_movies = []
    for mv in tqdm(movies):
        movie = Movie()
        movie.pk = mv['id']
        movie.title_en = mv['original_title']
        movie.title_ko = mv['title']
        movie.rate = mv['vote_average']
        movie.rate_people_count = mv['vote_count']
        movie.poster_path = mv['poster_path']
        movie.description = mv['overview']
        movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
        to_db_movies.append(movie)
        
    div = 100
    to_db_movies_batch = []
    for i in range(len(to_db_movies) // div + 1):
        to_db_movies_batch.append(to_db_movies[i*div:(i+1)*div])
        
    for idx, batch in enumerate(to_db_movies_batch):
        print(round(idx/len(to_db_movies_batch), 2), end='|')
        temp = Movie.objects.bulk_create(batch)
    # add genre
    mgs = []
    for mv in movies:
        for gid in mv['genre_ids']:
            mg = MG()
            mg.movie_id = mv['id']
            mg.genre_id = gid
            mgs.append(mg)
    temp = MG.objects.bulk_create(mgs)

In [39]:
store_movies(2000)

storing-----------2000


100%|███████████████████████████████████████████████████████████████████████| 5649/5649 [00:00<00:00, 65860.99it/s]


0.0|0.02|0.04|0.05|0.07|0.09|0.11|0.12|0.14|0.16|0.18|0.19|0.21|0.23|0.25|0.26|0.28|0.3|0.32|0.33|0.35|0.37|0.39|0.4|0.42|0.44|0.46|0.47|0.49|0.51|0.53|0.54|0.56|0.58|0.6|0.61|0.63|0.65|0.67|0.68|0.7|0.72|0.74|0.75|0.77|0.79|0.81|0.82|0.84|0.86|0.88|0.89|0.91|0.93|0.95|0.96|0.98|

In [ ]:
for year in reversed(range(2000, 2022)):
    store_movies(year)